In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# Encoder
encoder_inputs = Input(shape=(None, 256))  # Example input shape (timesteps, features)
encoder_lstm = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, 256))
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(256, activation='softmax')  # Example output shape
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
model.summary()


In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64, epochs=50, validation_split=0.2)


In [ ]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))


In [ ]:
# dataset gen

import numpy as np
import hashlib
import random
import string

# Generate a random word
def random_word(length=6):
    return ''.join(random.choices(string.ascii_lowercase, k=length))

# Generate SHA-256 hash
def sha256_hash(word):
    return hashlib.sha256(word.encode()).hexdigest()

# Create dataset
num_samples = 10000  # Adjust size
max_word_length = 6  # Fixed word length for simplicity

words = [random_word(max_word_length) for _ in range(num_samples)]
hashes = [sha256_hash(word) for word in words]

# Convert to character sequences (for Seq2Seq)
char_set = string.ascii_lowercase + string.digits  # Limited charset
char_to_index = {c: i for i, c in enumerate(char_set, 1)}  # Map chars to numbers
index_to_char = {i: c for c, i in char_to_index.items()}

# Encode words as sequences
def encode_word(word, max_length):
    return [char_to_index[c] for c in word] + [0] * (max_length - len(word))  # Padding

# Encode hashes (convert hex chars to numeric sequences)
def encode_hash(hash_str, max_length=64):  # SHA-256 is 64 hex chars
    return [char_to_index.get(c, 0) for c in hash_str] + [0] * (max_length - len(hash_str))

# Create numerical dataset
X = np.array([encode_hash(h) for h in hashes])
Y = np.array([encode_word(w, max_word_length) for w in words])

# Reshape for LSTM (batch_size, timesteps, features)
X = X.reshape((num_samples, 64, 1))  # 64-timestep hash input
Y = Y.reshape((num_samples, max_word_length, 1))  # word output


In [ ]:
#train 

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# Define Model
encoder_inputs = Input(shape=(64, 1))  # SHA-256 input
encoder = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_word_length, 1))
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(char_set) + 1, activation="softmax")  # Predict characters
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit([X, Y], Y, batch_size=64, epochs=50, validation_split=0.2)
